In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

In [ ]:
DATA_DIR='/kaggle/input/llm-detect-ai-generated-text/'
df_train_essays = pd.read_csv(DATA_DIR + "train_essays.csv")

In [ ]:
# df_train_essays_ext = pd.read_csv('/kaggle/input/llm-detect-ai-generated-vs-student-generated-text/LLM.csv')
# df_train_essays_ext.rename(columns = {"Label":"generated"}, inplace=True)
# df_train_essays_ext.rename(columns = {"Text":"text"}, inplace=True)
# df_train_essays_ext['generated']=df_train_essays_ext['generated'].replace({'ai':1,'student':0})


# df_train_essays_ext2 = pd.read_csv('/kaggle/input/daigt-v3-train-dataset/train_v3_drcat_01.csv')
# df_train_essays_ext2.rename(columns = {"label":"generated"}, inplace=True)


# df_train_essays_ext4 = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text-deobfuscation/train_from_LLM-Detect_AI-GT_1MNB-3SGD.csv')
# df_train_essays_ext4.rename(columns = {"label":"generated"}, inplace=True)

# df_train_essays_ext5 = pd.read_csv('/kaggle/input/gemini-pro-llm-daigt-dataset/train_essays_v1.csv')
# df_train_essays_ext5.rename(columns = {"label":"generated"}, inplace=True)

# df_train_essays_ext7 = pd.read_csv('/kaggle/input/daigt-one-place-all-data/concatenated.csv')
# df_train_essays_ext7.rename(columns = {"label":"generated"}, inplace=True)

# df_train_essays_final = pd.concat([df_train_essays[["text", "generated"]],df_train_essays_ext2[["text", "generated"]], df_train_essays_ext4[["text", "generated"]],df_train_essays_ext5[["text", "generated"]],df_train_essays_ext7[["text", "generated"]],df_train_essays_ext[["text", "generated"]]])

In [ ]:
# df_train_essays_final=df_train_essays_final.drop_duplicates(subset='text')

In [ ]:
# X=np.array(df_train_essays_final['text'])
# y=np.array(df_train_essays_final['generated'])

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(df_train_essays_final['generated'])

In [ ]:
# string_indices = np.where(np.array([isinstance(x, str) for x in X]))

# X_strings = X[string_indices]
# y_strings = y[string_indices]

# print( X_strings.shape)
# print( y_strings.shape)

In [ ]:
# indices=np.isnan(y_strings)
# X_=X_strings[~indices]
# y_=y_strings[~indices]


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_,
#                                                     y_,
#                                                     test_size=0.15,
#                                                     random_state=42)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
y_train=np.load('/kaggle/input/final-all-inputs/Approach_4/y_train.npy')
y_test=np.load('/kaggle/input/final-all-inputs/Approach_4/y_test.npy')
X_train=np.load('/kaggle/input/texts-data/X_train.npy',allow_pickle=True)
X_test=np.load('/kaggle/input/texts-data/X_test.npy',allow_pickle=True)

In [ ]:
# tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")

tokenizer=ElectraTokenizer.from_pretrained('/kaggle/input/final-all-inputs/Approach_4/tokenizer')

In [ ]:
p=[str(x) for x in X_train]

In [ ]:
encoded_inputs = tokenizer(p, padding=True, truncation=True, return_tensors="tf")

In [ ]:
# import pickle
# with open('encoded_inputs.pkl','wb') as f:
#     pickle.dump(encoded_inputs,f)

In [ ]:
labels = tf.convert_to_tensor(y_train)
dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": encoded_inputs["input_ids"], "attention_mask": encoded_inputs["attention_mask"]}, labels))

In [ ]:
# model = TFElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=2)


model=TFElectraForSequenceClassification.from_pretrained("/kaggle/input/final-all-inputs/Approach_4/model")

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [ ]:
batch_size = 32
train_dataset = dataset.shuffle(buffer_size=100).batch(batch_size)

In [ ]:
model.fit(train_dataset, epochs=3)


In [ ]:
t=[str(x) for x in X_test]

In [ ]:
encoded_test=tokenizer(t,padding=True,truncation=True, return_tensors="tf")

# with open("/kaggle/input/final-all-inputs/Approach_4/encoded_test.pkl",'rb') as f:
#     encoded_test=pickle.load(f)

In [ ]:
test_pred_logits = model.predict({"input_ids": encoded_test["input_ids"], "attention_mask": encoded_test["attention_mask"]})

In [ ]:
test_preds = tf.argmax(test_pred_logits.logits, axis=1).numpy()


In [ ]:
accuracy_score(test_preds,y_test)

In [ ]:
DATA_DIR='/kaggle/input/llm-detect-ai-generated-text/'
df_test_essays = pd.read_csv(DATA_DIR + "test_essays.csv")
final_test_data=np.array(df_test_essays['text'])


In [ ]:
t_=[str(x) for x in final_test_data]

In [ ]:
encoded_test=tokenizer(t,padding=True,truncation=True, return_tensors="tf")


In [ ]:
test_pred_logits = model.predict({"input_ids": encoded_test["input_ids"], "attention_mask": encoded_test["attention_mask"]})

In [ ]:
prob=tf.nn.softmax(test_pred_logits.logits, axis=-1)
probs=prob[:,1]
probs=probs.numpy()

In [ ]:
sub=df_test_essays.drop('prompt_id',axis=1)
submission=sub.drop('text',axis=1)
submission['generated']=probs


In [ ]:
# tokenizer.save_pretrained("tokenizer")
# model.save_pretrained("model")

In [ ]:
submission.to_csv('submission.csv',index=False)
submission